So this isn't working, but some ideas from the Rives paper might be worth trying
There they used a linear logistic regression layer to classify into 8 different secondary structure types, directly from the transformer output.
We could find all annotated, cofactor-mediated reactions, and use a similar output layer. 
Then the challenge would be to convert from one cofactor to another while preserving function..

In [1]:
from itertools import islice
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from fairseq.data import data_utils
from fairseq.models.roberta import RobertaModel
from scipy.sparse import load_npz
from sklearn.metrics import precision_recall_curve
from tqdm import tqdm

sys.path.append('..')
from go_annotation.ontology import Ontology

%matplotlib inline
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

roberta_model = RobertaModel.from_pretrained(
    '/projects/deepgreen/pstjohn/20210121_go_checkpoints/',
    data_name_or_path='/projects/deepgreen/pstjohn/swissprot_go_annotation/fairseq_swissprot/',
    checkpoint_file='swissprot_preinit.pt')

_ = roberta_model.eval()  # disable dropout (or leave in train mode to finetune)
_ = roberta_model.to(device) 

In [4]:
def encode(roberta_model, sequence):
    sequence = '<s> ' + ' '.join(sequence.replace('B', 'D').replace('Z', 'E').replace('J', 'L'))
    encoded_sequence = roberta_model.task.source_dictionary.encode_line(sequence, add_if_not_exist=False)[:roberta_model.model.max_positions()]    
    return encoded_sequence


def inputs_generator(sequences, batch_size):
    encoded_lines = (encode(roberta_model, line) for line in sequences)
    for batch in iter(lambda: tuple(islice(encoded_lines, batch_size)), ()):
        yield data_utils.collate_tokens(
            batch, pad_idx=roberta_model.task.source_dictionary.pad()).long().to(device)
        

def calc_features(protein_sequences, batch_size=8):
    features = []
    for inputs in tqdm(inputs_generator(protein_sequences, batch_size),
                       total=int(np.ceil(len(protein_sequences)/batch_size))):
        
        assert inputs.shape[1] <= 1024
        assert inputs.max() <= 25

        with torch.no_grad():
            features += [roberta_model.extract_features(inputs)[:, 0, :].detach().cpu().numpy()]

    return np.concatenate(features)

In [5]:
# Load swissprot sequences and annotations
import os
swissprot_dir = '/projects/deepgreen/pstjohn/swissprot_go_annotation'

swissprot = pd.read_parquet(os.path.join(swissprot_dir, 'parsed_swissprot_uniref_clusters.parquet'))

In [6]:
seq_df = pd.read_csv('genes_with_sequences.csv', index_col=0)
rxn_pairs = pd.read_csv('reaction_pairs.csv')
seq_df_dedup = seq_df.drop_duplicates(subset='protein_sequence', keep=False)

In [8]:
merged = rxn_pairs.merge(seq_df_dedup.reset_index(drop=True).reset_index(), left_on='rxnid_nad', right_on='reaction')\
    .merge(seq_df_dedup.reset_index(drop=True).reset_index(), left_on='rxnid_nadp', right_on='reaction')

merged_same_model = merged[merged.model_x == merged.model_y]
merged_same_model.head()

,rxnid_nad,rxnid_nadp,index_x,model_x,reaction_x,gene_id_x,protein_sequence_x,index_y,model_y,reaction_y,gene_id_y,protein_sequence_y
14,FMNRx,FMNRx2,174,iSB619,FMNRx,SA_RS04650,MKSKYEPLFDKVELPNGVELRNRFVLAPLTHISSNDDGTISDVELP...,743,iSB619,FMNRx2,SA_RS03020,MKGLIIIGSAQVNSHTSALARYLTEHFKTHDIEAEIFDLAEKPLNQ...
15,FMNRx,FMNRx2,174,iSB619,FMNRx,SA_RS04650,MKSKYEPLFDKVELPNGVELRNRFVLAPLTHISSNDDGTISDVELP...,744,iSB619,FMNRx2,SA_RS01880,MNIVLLSGSTVGSKTRIAMDDLKNELEVINEGHQIELMDLRELELE...
41,FMNRx,FMNRx2,176,iYS854,FMNRx,USA300HOU_RS04740,MKSKYEPLFDKVELPNGVELRNRFVLAPLTHISSNDDGTISDVELP...,750,iYS854,FMNRx2,USA300HOU_RS01800,MNIVLLSGSTVGSKTRIAMDDLKNELEVINEGHQIELMDLRELELE...
42,FLVRx,FLVR,177,iYS854,FLVRx,USA300HOU_RS01710,MYRYKPLLQSIHLPNGIKISNRFVLSPMTVNTSTKEGYITKEDLAY...,751,iYS854,FLVR,USA300HOU_RS13680,MSNMNQTIMDAFHFRHATKQFDPQKKVSKEDFETILESGRLSPSSL...
64,ALDD2x,ALDD2y,114,iJN1463,ALDD2x,PP_2589,MYTLEFWQQRASDLYLPAHALIDGKPVNAQDGATFAAINPATNNVL...,660,iJN1463,ALDD2y,PP_2492,MLNFDFHNPTRIIFGNQTIARLDELIPTEARILLLYGGQSARASGT...


In [25]:
swissprot_sample = swissprot.sample(10000)

In [26]:
to_append = seq_df_dedup[['gene_id', 'protein_sequence']].rename(columns={'gene_id': 'accession', 'protein_sequence': 'sequence'})
to_append['set'] = 'nad'

swissprot_df = swissprot_sample[['accession', 'sequence']].copy()
swissprot_df['set'] = 'swissprot'

seq_data = swissprot_df.append(to_append).drop_duplicates('sequence', keep='last')

In [27]:
features = calc_features(seq_data.sequence, 16)

100%|██████████| 699/699 [04:15<00:00,  2.73it/s]


In [28]:
from sklearn.decomposition import PCA

In [29]:
# pca = PCA()
# swissprot_pca = pca.fit_transform(np.vstack([swissprot_features, new_features]))

# # plt.semilogy(pca.explained_variance_ratio_)

# nad_pca = pca.transform(feat_nad)
# nadp_pca = pca.transform(feat_nadp)

# plt.plot(swissprot_pca[:, 0], swissprot_pca[:, 1], '.', color='k', alpha=.3, ms=1)
# plt.plot(nad_pca[:, 0], nad_pca[:, 1], '.')
# plt.plot(nadp_pca[:, 0], nadp_pca[:, 1], '.')

NameError: name 'swissprot_features' is not defined

In [56]:
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=(
    ('PCA', PCA(n_components=100)),
    ('NearestNeighbors', NearestNeighbors(n_neighbors=1))
))

pipe.fit(features)

Pipeline(steps=[('PCA', PCA(n_components=100)),
                ('NearestNeighbors', NearestNeighbors(n_neighbors=1))])

In [57]:
def pipe_kneighbors(pipe, X):
    Xt = pipe.steps[0][-1].transform(X)
    return pipe.steps[-1][-1].kneighbors(Xt)

In [58]:
feat_nad = calc_features(merged_same_model.protein_sequence_x, 16)
feat_nadp = calc_features(merged_same_model.protein_sequence_y, 16)

100%|██████████| 3/3 [00:00<00:00,  4.11it/s]


In [59]:
distances, indices = pipe_kneighbors(pipe, feat_nad + np.median(feat_nadp - feat_nad, axis=0))

In [60]:
df = pd.DataFrame(indices)
df['reaction_nad'] = merged_same_model.reaction_x.values
df['reaction_nadp'] = merged_same_model.reaction_y.values

In [72]:
seq_data

,accession,sequence,set
208,C3LNM7,MQVTVETLEGLQRRLNITVPAANIEDAVAAELRNIAKNRRFDGFRK...,swissprot
277,A4VYP1,MANKKIRIRLKAYEHRTLDTAAAKIVETATRTGAQVAGPVPLPTER...,swissprot
201,Q6ENW5,MAKKSLIQREKKRHKLEQKYHLIRRSSKKKIRSKVSPLSLSEKTKM...,swissprot
86,P0DC03,MITGKLITVEGPDGAGKTTVLEQLIPLLKQKVAQEILTTREPGGVA...,swissprot
113,Q2SSQ6,MIKKIGILTSGGDAPGMNNAIAGVVKTASSKGIEVYGINDGYKGLV...,swissprot
...,...,...,...
11304,ECD_RS16020,MLYDKSLERDNCGFGLIAHIEGEPSHKVVRTAIHALARMQHRGAIL...,nad
11309,ABAYE_RS02030,MAERLNNDFQFLDVPRQDPEKKDITVRKAEFVEIYKPFTSETVTNQ...,nad
11310,ABAYE_RS02025,MHMPSPNTVAPAQGLYQPDEFKDNCGFGLIAHMKGESSHHLVETAI...,nad
11320,ECOLC_RS02600,MLYDKSLERDNCGFGLIAHIEGEPSHKVVRTAIHALARMQHRGAIL...,nad


In [70]:
seq_data.iloc[10051]

accession                                    USA300HOU_RS01710
sequence     MYRYKPLLQSIHLPNGIKISNRFVLSPMTVNTSTKEGYITKEDLAY...
set                                                        nad
Name: 2055, dtype: object

In [76]:
df.melt(id_vars=['reaction_nad', 'reaction_nadp']).merge(seq_data.reset_index()['accession'], left_on='value', right_index=True, how='left')

,reaction_nad,reaction_nadp,variable,value,accession
0,FMNRx,FMNRx2,0,10050,USA300HOU_RS04740
1,FMNRx,FMNRx2,0,10050,USA300HOU_RS04740
2,FMNRx,FMNRx2,0,10050,USA300HOU_RS04740
3,FLVRx,FLVR,0,10051,USA300HOU_RS01710
4,ALDD2x,ALDD2y,0,9988,PP_2589
5,FDH,FDH6r,0,10092,CD630_33170
6,FDH,FDH6r,0,10092,CD630_33170
7,MTHFD2i,MTHFD,0,10080,SGL_RS12965
8,ALCD2x,ALCD2y,0,10202,CLJU_RS05830
9,ME1,ME2,0,10255,KPN_01847


In [41]:
merged_same_model.reaction_x

14        FMNRx
15        FMNRx
41        FMNRx
42        FLVRx
64       ALDD2x
191         FDH
192         FDH
372     MTHFD2i
681      ALCD2x
798         ME1
893         ME1
1042        ME1
1043        ME1
1066        ME1
1067        ME1
1095        ME1
1117        ME1
1209        ME1
1222        ME1
1337        ME1
1431      NADH5
1432      NADH5
1707     GLUDxi
1708     GLUDxi
1741     GLUDxi
1742     GLUDxi
1873     GLUDxi
1874     GLUDxi
1935     GLUDxi
1936     GLUDxi
2884       RDH1
3134      HACD1
3135      HACD1
3137      HACD1
3138      HACD1
3140      HACD1
3141      HACD1
Name: reaction_x, dtype: object

In [38]:
indices

array([[10050, 10048, 10051, 10538,  1329],
       [10050, 10048, 10051, 10538,  1329],
       [10050, 10048, 10051, 10538,  1329],
       [10051, 10048, 10050,  7492, 10612],
       [ 9988,  9993,  9994, 10177,  9981],
       [10092, 10099, 10106, 10103, 10102],
       [10092, 10099, 10106, 10103, 10102],
       [10080,  2546,  8945,   424,   418],
       [10202, 10007, 10614, 10613,  9901],
       [10255, 10251, 10243, 10240, 10239],
       [10255, 10251, 10243, 10240, 10239],
       [10257, 10248, 10249, 10254, 10253],
       [10257, 10248, 10249, 10254, 10253],
       [10257, 10249, 10248, 10254, 10800],
       [10257, 10249, 10248, 10254, 10800],
       [10255, 10251, 10243, 10240, 10239],
       [10255, 10251, 10243, 10240, 10239],
       [10255, 10251, 10243, 10240, 10239],
       [10255, 10251, 10243, 10240, 10239],
       [10255, 10247, 10251, 10244, 10243],
       [10287, 10282, 10284, 10833, 10829],
       [10287, 10282, 10284, 10833, 10829],
       [10434, 10435, 10429, 104

In [37]:
distances

array([[1.52018628, 1.53159553, 3.09589415, 4.61694822, 4.69137506],
       [1.52018628, 1.53159553, 3.09589415, 4.61694822, 4.69137506],
       [1.53182989, 1.54676018, 3.13211347, 4.60756689, 4.72700117],
       [1.52880643, 3.15969861, 3.18246492, 4.10788625, 4.13409645],
       [1.53071445, 2.78137114, 3.13046138, 3.47331272, 3.5073463 ],
       [1.52117932, 2.05514704, 3.25043865, 3.28009685, 3.30794978],
       [1.52117932, 2.05514704, 3.25043865, 3.28009685, 3.30794978],
       [1.52572075, 3.87424069, 4.20017864, 4.34836165, 4.56795123],
       [1.52253656, 1.68412108, 2.31993242, 2.39574494, 3.19718057],
       [1.44073402, 1.48591516, 1.49924402, 1.50072568, 1.50567881],
       [1.4373982 , 1.4811315 , 1.49385323, 1.49531317, 1.49936751],
       [1.51185872, 1.52869147, 1.54205255, 1.58331263, 1.80389856],
       [1.51185872, 1.52869147, 1.54205255, 1.58331263, 1.80389856],
       [1.50717889, 1.52861215, 1.55520408, 1.60780618, 1.75581589],
       [1.50717889, 1.52861215, 1.